In [1]:
import pandas as pd
import numpy as np

market = pd.read_csv("data/processed/market_snapshot.csv")
dist = pd.read_csv("data/processed/distribution_mvp_etherscan.csv")

df = market.merge(
    dist[["coingecko_id","verified_contract","contract_address"]],
    on="coingecko_id",
    how="left"
)

df.head()


,coingecko_id,pulled_at_utc,current_price_usd,market_cap_usd,volume_24h_usd,circulating_supply,max_supply,fdv_usd,token_name_x,symbol_x,...,tier_y,category_y,chain_y,token_name,symbol,tier,category,chain,verified_contract,contract_address
0,uniswap,2026-01-19T12:34:30.906620+00:00,4.980000,3.160631e+09,382896158,6.346104e+08,1.000000e+09,4480843112,Uniswap,UNI,...,A,DEX,ethereum,Uniswap,UNI,A,DEX,ethereum,True,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
1,chainlink,2026-01-19T12:34:45.019802+00:00,12.750000,9.024027e+09,648920230,7.081000e+08,1.000000e+09,12744001030,Chainlink,LINK,...,A,Infrastructure & Middleware,ethereum,Chainlink,LINK,A,Infrastructure & Middleware,ethereum,True,0x514910771af9ca656af840dff83e8264ecf986ca
2,aave,2026-01-19T12:34:47.605564+00:00,163.460000,2.481823e+09,375268933,1.518357e+07,1.600000e+07,2615272322,Aave,AAVE,...,A,Lending & Borrowing,ethereum,Aave,AAVE,A,Lending & Borrowing,ethereum,True,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
3,lido-dao,2026-01-19T12:35:14.316675+00:00,0.546995,4.629709e+08,56720533,8.465668e+08,1.000000e+09,546880520,Lido Dao,LDO,...,A,Liquid staking,ethereum,Lido Dao,LDO,A,Liquid staking,ethereum,True,0x5a98fcbea516cf06857215779fd812ca3bef1b32
4,curve-dao-token,2026-01-19T12:35:17.069355+00:00,0.390634,5.737410e+08,74930533,1.468781e+09,3.030303e+09,915416077,Curve DAO,CRV,...,B,DEX,ethereum,Curve DAO,CRV,B,DEX,ethereum,True,0xd533a949740bb3306d119cc777fa900ba034cd52


In [2]:
df["circ_to_max_ratio"] = df["circulating_supply"] / df["max_supply"]
df["fdv_to_mcap_ratio"] = df["fdv_usd"] / df["market_cap_usd"]

df[["coingecko_id","circ_to_max_ratio","fdv_to_mcap_ratio"]].head()


,coingecko_id,circ_to_max_ratio,fdv_to_mcap_ratio
0,uniswap,0.634610,1.417705
1,chainlink,0.708100,1.412230
2,aave,0.948973,1.053770
3,lido-dao,0.846567,1.181242
4,curve-dao-token,0.484698,1.595521


In [3]:
def percentile_score(series, higher_is_better=True):
    pct = series.rank(pct=True)
    if not higher_is_better:
        pct = 1 - pct
    return (pct * 100).clip(0, 100)

# Higher circ/max is better
df["circulation_score"] = percentile_score(
    df["circ_to_max_ratio"], higher_is_better=True
)

# Lower FDV/MCAP is better
df["fdv_pressure_score"] = percentile_score(
    df["fdv_to_mcap_ratio"], higher_is_better=False
)


In [4]:
df["mint_risk_score"] = np.where(
    df["verified_contract"] == True,
    70,   # neutral baseline
    30    # penalty for opacity
)


In [5]:
df["max_supply_certainty"] = np.where(
    df["max_supply"].notna() & (df["max_supply"] > 0),
    100,
    40
)


In [6]:
df["supply_sustainability_score"] = (
    0.35 * df["circulation_score"] +
    0.30 * df["fdv_pressure_score"] +
    0.20 * df["mint_risk_score"] +
    0.15 * df["max_supply_certainty"]
).clip(0, 100)


In [7]:
df.to_csv("data/processed/supply_sustainability_scores.csv", index=False)

df[["coingecko_id","supply_sustainability_score"]].sort_values(
    "supply_sustainability_score", ascending=False
)


,coingecko_id,supply_sustainability_score
12,pepe,91.000000
5,havven,84.166667
11,basic-attention-token,78.833333
6,compound-governance-token,75.000000
10,yearn-finance,71.166667
2,aave,65.833333
16,1inch,62.000000
3,lido-dao,56.666667
7,balancer,53.500000
1,chainlink,52.833333
